# Capstone: Project 1
The code cell provided below is for the reference code to unzip the unput dataset on your local system.

#### Note: We do not recommend running the code in the lab environment. The zip file size will delay the code execution and may lead to some unforseen errors. The input files have already been unzipped for use in this code.

In [20]:
#import shutil
#shutil.unpack_archive("Images.zip")

#### Step 1: Data Loading and Preprocessing

In [10]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
#### Step 2: Load and preprocess the data

# Load the labels from labels.csv
labels_df = pd.read_csv('labels.csv', sep=',', header=None)
labels_df.columns = ['image_id', 'class', 'x_min', 'y_min', 'x_max', 'y_max']

# Adjust the image IDs in the dataframe
labels_df['image_id'] = labels_df['image_id'].apply(lambda x: f"{x:08d}")

# Use iloc to pick the first 1000 labels
labels_df = labels_df.iloc[:1000]

# Load the corresponding images
images_dir = 'Images/'
images = []
for index, row in labels_df.iterrows():
    img_path = os.path.join(images_dir, f"{row['image_id']}.jpg")
    img = cv2.imread(img_path)
    if img is not None:
        images.append(img)
    else:
        print(f"Error loading image: {img_path}")

images = np.array(images)

# Check if images are loaded
if len(images) == 0:
    print("No images loaded. Please check the image paths.")
else:
    print(f"{len(images)} images loaded successfully.")

1000 images loaded successfully.


/tmp/ipykernel_63/2335160996.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  images = np.array(images)


#### Step 2: Exploratory Data Analysis (EDA)

In [ ]:
# Analyze the distribution of vehicle types in the limited dataset


In [ ]:
# Address data quality issues arising from the discrepancy between labels and actual image filenames
# Sorting the image filenames


#### Step 3: Preprocess the images

In [ ]:
if len(images) > 0:
    processed_images = [cv2.resize(img, (224, 224)) for img in images]  # Adjust dimensions as needed
    processed_images = np.array(processed_images)
    print("Images resized successfully.")

#### Step 4:  Prepare the labels and bounding boxes

In [ ]:
labels = labels_df['class'].to_numpy()
bounding_boxes = labels_df[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()

# Convert labels to one-hot encoding


#### Step 5: Split the data into training and testing sets

In [ ]:
X_train, X_test, y_train, y_test, bbox_train, bbox_test = train_test_split(processed_images, labels, bounding_boxes, test_size=0.2, random_state=42)

#### Step 6: Model Creation and Training

In [ ]:
def create_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    vehicle_class = layers.Dense(num_classes, activation='softmax', name='vehicle_class')(x)
    bounding_box = layers.Dense(4, name='bounding_box')(x)

    model = keras.Model(inputs=inputs, outputs=[vehicle_class, bounding_box])
    return model

input_shape = processed_images[0].shape
num_classes = len(unique_labels)
model = create_model(input_shape, num_classes)


model.compile(optimizer='adam',
              loss={'vehicle_class': 'sparse_categorical_crossentropy', 'bounding_box': 'mse'},
              metrics={'vehicle_class': 'accuracy', 'bounding_box': 'mae'})

model.fit(X_train, {'vehicle_class': y_train, 'bounding_box': bbox_train}, epochs=25, validation_data=(X_test, {'vehicle_class': y_test, 'bounding_box': bbox_test}))

#### Step 7: Model Evaluation

In [ ]:
test_results = model.evaluate(X_test, {'vehicle_class': y_test, 'bounding_box': bbox_test}, verbose=2)
print('\nTest results:', test_results)

#### Step 8: Inferencing and Visualization

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Choose a few sample images for inference


In [ ]:
# Perform inference on the sample images


In [ ]:
# Extract the predicted bounding box coordinates


In [ ]:
# Visualize the sample images with predicted bounding boxes
